![IFMG](https://images.even3.com.br/sBWZnWPFUBgLOGciSZc4G5ZQy7Q=/1100x440/smart/even3.blob.core.windows.net/banner/ARTEPARASITE3.993b3db9f908426e9833.png)

---

# Introdução ao Tensorflow - Parte 1

---

#### Professor: Felipe Reis

#### Data: 20-10-2021

---
### Informações Iniciais

Este tutorial contém um conjunto de códigos-fonte que podem ser executados com auxílio do [Jupyter](https://jupyter.org/) ou do [Google Colab](http://colab.research.google.com/). 

Os códigos fontes foram desenvolvidos com base na documentação do [Tensorflow](https://www.tensorflow.org/tutorials?hl=pt-br) e do [Keras](https://keras.io/api/).

Esta primeira parte foi desenvolvida com base no tutorial "*Treine sua primeira rede neural: classificação básica*", disponibilizado na documentação do [Tensorflow](https://www.tensorflow.org/tutorials/keras/classification?hl=pt-br).

Boa prática.

*Felipe A. L. Reis*

---
### Importação de bibliotecas 

In [ ]:
!pip install graphviz
!pip install pydot
!pip install pydotplus

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import random
import time

#import plot_images
from xject_helper import xject_helper as helper

#tensorflow
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow import keras

---
### Uso de GPUs

Para maior velocidade no treinamento e uso de redes neurais, verifique se está utilizando GPUs.

Caso esteja utilizando o [Google Colab](http://colab.research.google.com/) e não tenha habilitado o uso de GPUs, siga a recomendação abaixo.

In [ ]:
if tf.test.gpu_device_name():
    print('Default GPU Name: {}'.format(device_lib.list_local_devices()))
else:
    print("Please install GPU version of TF")
    print('Acesse o menu Edit > Notebook Settings')
    print('Escolha a opção "Hardware Accelerator: GPU"')
    print('Importe novamente as bibliotecas e execute o código novamente')

In [ ]:
#Impressão de versão do Tensorflow
print('Versão Tensorflow:', tf.__version__)
print('Versão Keras:', keras.__version__)

---
## Base de Dados MNIST

O banco de dados MNIST contém dígitos manuscritos (escritos a mão).

Possui um conjunto de treinamento com 60.000 imagens e um conjunto de teste com 10.000 imagens. 

Os dígitos foram normalizados por tamanho e centralizados em uma imagem de tamanho fixo.

A base de dados foi criada por Yann LeCun em 1998.

Ela está disponível em: [http://yann.lecun.com/exdb/mnist/](http://yann.lecun.com/exdb/mnist/)

![MNist](https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png)

Fonte: *Yann LeCun, Corinna Cortes, Christopher J.C. Burges. **Gradient-based learning applied to document recognition**. 1998. Proceedings of the IEEE, 86(11):2278-2324. Disponível em: http://yann.lecun.com/exdb/mnist/. Acesso em: 11 de outubro de 2021.*

In [ ]:
#download da base de dados
mnist = keras.datasets.mnist

#divisão em base de treinamento e testes (subdividido em imagens e labels)
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
#definição das classes 
class_names = ['Dígito 0', 'Dígito 1', 'Dígito 2', 'Dígito 3', 'Dígito 4', 
               'Dígito 5', 'Dígito 6', 'Dígito 7', 'Dígito 8', 'Dígito 9']

In [ ]:
#informações sobre o tamanho o vetor das imagens (6k imagens, de 28x28)
train_images.shape

In [ ]:
#imprime os labels de treinamento (nome das classes)
train_labels

In [ ]:
#informações sobre o tamanho o vetor das labels (6k labels)
print(train_labels.shape)

In [ ]:
#podemos visualizar algumas imagens do conjunto de treinamento
helper.plot_images(train_images, train_labels, class_names, rows=5, columns=8, binary=True, random=False)

### Pré processamento

Para melhor aprendizar da rede, é necessário normalizar as imagens.

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
#podemos visualizar algumas imagens do conjunto de treinamento
helper.plot_images(train_images, train_labels, class_names, rows=5, columns=8, binary=True, random=False)

### Construção da rede neural

O principal bloco de construção da rede neural é a camada (*layer*). As camadas  extraem representações dos dados inseridos na rede. Na construção de redes, algumas camadas podem ser encadeadas

#### Modelo Sequencial

O modelo sequencial é adequado para redes neurais simples, onde cada camada tem exatamente um [tensor](https://pt.wikipedia.org/wiki/Tensor) de entrada e um [tensor](https://pt.wikipedia.org/wiki/Tensor) de saída.

Para usar um modelo de rede sequencial, é necessário utilizar o comando `keras.Sequential`.

Mais informações: https://keras.io/guides/sequential_model/

#### Camada Flatten

*Flatten* é uma camada utilizada para achatar o vetor de entrada.

Ex.: 

* Entrada: *(None, 10, 32)* 
* Saída: *(None, 320)*

Para usar uma camada *Flatten*, é necessário utilizar o comando `keras.layers.Flatten`.

Mais informações: https://keras.io/api/layers/reshaping_layers/flatten/


#### Camada Dense

As camadas do tipo `tf.keras.layers.Dense` contém neurônios totalmente conectados. 

Essas camadas correspondem a um conjunto convencional de neurônios artificiais, com entrada, produto de pesos e entradas, funções de ativação e saída.

$$ x_j = g \left( \sum_{i=1}^{n} (x_i \cdot w_i) + w_b \right) $$

Os pesos desses neurônios são aprendidos durante o treinamento.

Essa camada possui diversos parâmetros, que podem ser utilizados para configuração da rede neural.

Para usar uma camada *Dense*, é necessário utilizar o comando `keras.layers.Dense`.

Mais informações: https://keras.io/api/layers/core_layers/dense/

In [ ]:
#modelo de rede sequencial
model = keras.Sequential([
    #transforma a image em um array de imagens de duas dimensões (of 28 by 28 pixels)
    keras.layers.Flatten(input_shape=(28, 28)), 
    
    #Camada de dados totalmente conectadas, com ativação relu
    #A camada possui 128 nós (neurônios)
    keras.layers.Dense(128, activation='relu'),
    
    #Camada de dados totalmente conectadas, com ativação softmax
    #A camada possui 10 nós (neurônios), correspondentes às probabilidades de cada classe (10 classes)
    keras.layers.Dense(10, activation='softmax')
])

#### Visualização Gráfica da Rede

O comando abaixo permite visualizar graficamente o modelo de rede neural.

In [ ]:
keras.utils.plot_model(
    model,
    show_shapes=True,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=True,
    dpi=96,
)

### Compilação do modelo

Antes do modelo estar pronto para o treinamento, são necessárias algumas configurações. 

Essas configurações são feitas no passo de compilação:

* **Função de Perda** (*Loss Function*)
  * Mede qual a diferença entre os valores preditos e previstos pelo modelo.
  * A função de perda deve ser minimizada, para que seja possível treinar a rede, de modo com que ela possa aprender e ter o menor erro.


* **Otimizador** (*Optimizer*)
  * Técnica para encontrar o ponto mínimo da função de perda, como o método do gradiente.


* **Métricas** (*Metrics*)
  * São medidas usadas para monitorar os passos de treinamento e teste. 
  * Fornecem uma informação estatística a respeito da qualidade dos resultados (intermediários ou finais).


Para compilar o modelo é necessário utilizar o comando `model.compile`.
  
Mais informações: https://keras.io/api/models/model_training_apis/

In [ ]:
#compila o modelo
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Treinamento da rede

Treinar a rede neural requer os seguintes passos:

1. Indicar as imagens de treinamento;
2. Informar os rótulos (*labels*), correspondente às classes das imagens
3. Definir o número de épocas.

Para começar a treinar, utilize o método `model.fit`.

Mais informações: https://keras.io/api/models/model_training_apis/#fit-method

In [ ]:
#realiza o treinamento da rede
model.fit(train_images, train_labels, epochs=5)

### Predições de imagens

Após treinado, é possível realizar previsões acerca de cada uma das imagens.

O método recebe uma imagem como entrada e produz uma classificação como saída.

Para realizar a classificação é utilizado o comando `model.predict`.

Mais informações: https://keras.io/api/models/model_training_apis/#predict-method

In [ ]:
#retorna predições de todo o conjunto de imagens de teste
predictions = model.predict(test_images)

#define a imagem a ser predita
num_imagem = 0
prediction = predictions[num_imagem]

#imprime a classe com maior probabilidade
class_pred = np.argmax(prediction)
print('Classe predita: ', class_names[class_pred])

In [ ]:
#imprime a probabilidade de cada classe para uma dada imagem
for cl, prob in zip(class_names, prediction):
    print(cl + ': {0:.2f}%'.format(prob*100))

### Visualização das Predições

Imprime a classe prevista (Prev) e a classe esperada (True).

O gráfico contém as probabilidades de cada uma das classes (dígitos de 0 a 9).

In [ ]:
helper.plot_pred(class_names, predictions, test_labels, test_images)

### Avaliação do modelo

Após treinado, é possível analisar a qualidade das previsões da rede neural, no conjunto de testes.

Avalia-se a perda e as métricas, de modo que seja possível analisar o desempenho da rede.

Este conjunto é utilizado como uma representação de uma situação real e não deve ser usado para treinamento.

A avaliação do conjunto é feita com o comando `model.evaluate`.

Mais informações: https://keras.io/api/models/model_training_apis/#evaluate-method

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print('Acurácia de testes: %.4f', test_acc)

---
---
# Tarefas

## Tarefa 1 (15 minutos)

Altere o modelo de rede neural abaixo, adicionando novas camadas e verifique o desempenho da rede.

Observe os efeitos da adição de camadas no tempo de treinamento e no desempenho global da rede.

Para isso, adicione mais camadas, utilizando o comando `keras.layers.Dense`.

Dica: Não adicione muitas camadas.

In [ ]:
#modelo de rede sequencial
model = keras.Sequential([
    #transforma a image em um array de imagens de duas dimensões (of 28 by 28 pixels)
    keras.layers.Flatten(input_shape=(28, 28)), 
    
    #Camada de dados totalmente conectadas, com ativação relu
    #A camada possui 128 nós (neurônios)
    keras.layers.Dense(128, activation='relu'),
    
    #Camada de dados totalmente conectadas, com ativação softmax
    #A camada possui 10 nós (neurônios), correspondentes às probabilidades de cada classe (10 classes)
    keras.layers.Dense(10, activation='softmax')
])

Visualize a arquitetura de rede neural criada.

In [ ]:
#imprime a imagem do modelo
keras.utils.plot_model(
    model,
    show_shapes=True,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=True,
    dpi=96,
)

Compile, treine e avalie a eficiência da arquitetura de rede criada.

In [ ]:
#compila o modelo
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#realiza o treinamento da rede
model.fit(train_images, train_labels, epochs=10)

In [ ]:
#avaliação do modelo, para o conjunto de testes
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('Acurácia de testes: %.4f', test_acc)

---
## Tarefa 2 (20 minutos)

Treine a rede neural criada na Tarefa 1 para o conjunto de dados abaixo.

### Base de Dados Fashion MNist

O banco de dados Fashion MNIST contém imagens de roupas, nas seguintes categorias.

0. Camiseta (*T-shirt / Top*)
1. Calça (*Trouser*)
2. Suéter (*Pullover*)
3. Vestido (*Dress*)
4. Casaco (*Coat*)
5. Sandália (*Sandal*)
6. Camisa (*Shirt*)
7. Tênis (*Sneaker*)
8. Bolsa (*Bag*)
9. Bota (*Ankle boot*)

Contém  60.000 imagens de treinamento e 10.000 imagens de teste. 
Os dígitos foram normalizados por tamanho e centralizados em uma imagem de tamanho fixo.

A base de dados foi criada pela Zalando Research com objetivo de ser um substituto direto para o conjunto de dados MNIST original. Essa base pode ser usada para benchmarking de algoritmos de aprendizado de máquina.

A base de dados tem licensa MIT e está disponível em [https://github.com/zalandoresearch/fashion-mnist](https://github.com/zalandoresearch/fashion-mnist)

A base de dados também pode ser encontrada no Kaggle: [https://www.kaggle.com/zalando-research/fashionmnist](https://www.kaggle.com/zalando-research/fashionmnist)

![Fashion MNist](https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/doc/img/fashion-mnist-sprite.png)

Fonte: *Zalando Research*. **A MNIST-like fashion product database**. 2017. Disponível em: https://github.com/zalandoresearch/fashion-mnist. Acesso em: 11 de outubro de 2021.*

In [ ]:
#download da base de dados
fashion_mnist = keras.datasets.fashion_mnist

#divisão em base de treinamento e testes (subdividido em imagens e labels)
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
#insira seu código aqui